In [2]:
import sys
import os
from awsglue.context import GlueContext
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
import subprocess

# Define parameters in a dictionary
parameters = {
    'JOB_NAME': 'FullLoadAndCDCProcesserJob',
    'RAW_S3_PATH': 's3://ecommerce-data-lake-us-east-1-dev/01_raw/',
    'STAGING_S3_PATH': 's3://ecommerce-data-lake-us-east-1-dev/02_staging/',
    'PREPROCESSED_S3_PATH': 's3://ecommerce-data-lake-us-east-1-dev/03_preprocessed/',
    'MASTER_S3_PATH': 's3://ecommerce-data-lake-us-east-1-dev/04_master/',
    'CURATED_S3_PATH': 's3://ecommerce-data-lake-us-east-1-dev/06_curated/',
    'TABLE_NAME': 'orders',
    'PROCESS_TYPE': 'full_load',
    'SOURCE': 'netSuite'
}

# Mock function to simulate getResolvedOptions
def get_resolved_options(args, keys):
    return {key: parameters[key] for key in keys}

# Mock sys.argv to simulate Glue job arguments
sys.argv = [
    '--JOB_NAME', parameters['JOB_NAME'],
    '--RAW_S3_PATH', parameters['RAW_S3_PATH'],
    '--STAGING_S3_PATH', parameters['STAGING_S3_PATH'],
    '--PREPROCESSED_S3_PATH', parameters['PREPROCESSED_S3_PATH'],
    '--MASTER_S3_PATH', parameters['MASTER_S3_PATH'],
    '--CURATED_S3_PATH', parameters['CURATED_S3_PATH'],
    '--TABLE_NAME', parameters['TABLE_NAME'],
    '--PROCESS_TYPE', parameters['PROCESS_TYPE'],
    '--SOURCE', parameters['SOURCE']
]

# Use the mock get_resolved_options function
args = get_resolved_options(sys.argv, [
    'JOB_NAME', 
    'RAW_S3_PATH', 
    'STAGING_S3_PATH', 
    'PREPROCESSED_S3_PATH', 
    'MASTER_S3_PATH',
    'CURATED_S3_PATH', 
    'TABLE_NAME', 
    'PROCESS_TYPE',
    'SOURCE'
])

# Set environment variables
os.environ['SPARK_VERSION'] = '3.1'

# Path to the Iceberg JAR file
iceberg_jar_path = "/opt/glue/jars/iceberg-spark3-runtime-0.12.0.jar"

# Path to the PyDeequ JAR file
pydeequ_jar_path = "/opt/glue/jars/deequ-glue-1.0-SNAPSHOT-jar-with-dependencies.jar"

# Initialize Spark session with Iceberg configurations
spark = SparkSession.builder \
    .config("spark.jars", iceberg_jar_path) \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog") \
    .config("spark.sql.catalog.spark_catalog.type", "hive") \
    .config("spark.sql.catalog.master_catalog", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.master_catalog.type", "hadoop") \
    .config("spark.sql.catalog.master_catalog.warehouse", "s3://ecommerce-data-lake-us-east-1-dev/04_master/") \
    .config("spark.sql.catalog.curated_catalog", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.curated_catalog.type", "hadoop") \
    .config("spark.sql.catalog.curated_catalog.warehouse", "s3://ecommerce-data-lake-us-east-1-dev/06_curated/") \
    .getOrCreate()

glueContext = SparkSession.builder.getOrCreate()
#GlueContext(SparkContext.getOrCreate())
print(args)


{'JOB_NAME': 'FullLoadAndCDCProcesserJob', 'RAW_S3_PATH': 's3://ecommerce-data-lake-us-east-1-dev/01_raw/', 'STAGING_S3_PATH': 's3://ecommerce-data-lake-us-east-1-dev/02_staging/', 'PREPROCESSED_S3_PATH': 's3://ecommerce-data-lake-us-east-1-dev/03_preprocessed/', 'MASTER_S3_PATH': 's3://ecommerce-data-lake-us-east-1-dev/04_master/', 'CURATED_S3_PATH': 's3://ecommerce-data-lake-us-east-1-dev/06_curated/', 'TABLE_NAME': 'orders', 'PROCESS_TYPE': 'full_load', 'SOURCE': 'netSuite'}


In [3]:
# Install Deequ
subprocess.check_call([sys.executable, "-m", "pip", "install", "--target=/tmp", "pydeequ"])
sys.path.insert(0, '/tmp')

  Using cached pydeequ-1.3.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached numpy-2.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached pandas-2.2.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl.metadata (1.8 kB)
Using cached pydeequ-1.3.0-py3-none-any.whl (37 kB)
Using cached numpy-2.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.3 MB)
Using cached pandas-2.2.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.0 MB)
Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl (229 kB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
Using cached six-1.16.0-p

In [4]:
from pydeequ.checks import Check, CheckLevel
from pydeequ.verification import VerificationSuite

In [5]:
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.sql.functions import col, lit, current_date, expr, when
from pyspark.sql.window import Window
import pyspark.sql.functions as F